In [4]:
# ------------------------------
# Online Payment Fraud Detection
# Decision Tree and Random Forest from scratch (No sklearn)
# ------------------------------

import pandas as pd
import numpy as np
from collections import Counter
import random

# ----------------------
# 1. Load dataset
# ----------------------
df = pd.read_csv("onlinefraud.csv")  # put CSV in working directory
print("Dataset shape:", df.shape)
print(df.head())

# ----------------------
# 2. Preprocessing
# ----------------------
target = "fraud_reported"
X = df.drop(columns=[target])
y = df[target].apply(lambda x: 1 if x=='Y' else 0).values  # convert to 0/1

# Convert categorical features to integers
for col in X.select_dtypes(include='object').columns:
    X[col] = pd.factorize(X[col])[0]

# Fill missing numeric values with median
X = X.fillna(X.median())
X = X.values

# ----------------------
# 3. Train/Test split (from scratch)
# ----------------------
def train_test_split_custom(X, y, test_size=0.2, random_state=None):
    if random_state:
        np.random.seed(random_state)
    indices = np.arange(X.shape[0])
    np.random.shuffle(indices)
    split = int(len(X)*(1-test_size))
    train_idx = indices[:split]
    test_idx = indices[split:]
    return X[train_idx], X[test_idx], y[train_idx], y[test_idx]

X_train, X_test, y_train, y_test = train_test_split_custom(X, y, test_size=0.2, random_state=42)

# ----------------------
# 4. Decision Tree (ID3)
# ----------------------
def entropy(y):
    counts = np.bincount(y)
    probs = counts / len(y)
    return -np.sum([p*np.log2(p) for p in probs if p>0])

def information_gain(y, y_left, y_right):
    return entropy(y) - (len(y_left)/len(y)*entropy(y_left) + len(y_right)/len(y)*entropy(y_right))

class Node:
    def __init__(self, predicted_class=None):
        self.feature_idx = None
        self.threshold = None
        self.left = None
        self.right = None
        self.predicted_class = predicted_class

class DecisionTree:
    def __init__(self, max_depth=None, min_samples_split=2):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.root = None
    
    def fit(self, X, y):
        self.root = self._grow_tree(X, y)
    
    def _grow_tree(self, X, y, depth=0):
        num_samples_per_class = [np.sum(y==i) for i in np.unique(y)]
        predicted_class = np.argmax(num_samples_per_class)
        node = Node(predicted_class=predicted_class)
        
        if depth < self.max_depth and len(y) >= self.min_samples_split:
            idx, thr = self._best_split(X, y)
            if idx is not None:
                indices_left = X[:, idx] <= thr
                X_left, y_left = X[indices_left], y[indices_left]
                X_right, y_right = X[~indices_left], y[~indices_left]
                node.feature_idx = idx
                node.threshold = thr
                node.left = self._grow_tree(X_left, y_left, depth+1)
                node.right = self._grow_tree(X_right, y_right, depth+1)
        return node
    
    def _best_split(self, X, y):
        best_gain = -1
        split_idx, split_thr = None, None
        for idx in range(X.shape[1]):
            thresholds = np.unique(X[:, idx])
            for thr in thresholds:
                y_left = y[X[:, idx] <= thr]
                y_right = y[X[:, idx] > thr]
                if len(y_left)==0 or len(y_right)==0:
                    continue
                gain = information_gain(y, y_left, y_right)
                if gain > best_gain:
                    best_gain = gain
                    split_idx = idx
                    split_thr = thr
        return split_idx, split_thr
    
    def predict(self, X):
        return np.array([self._predict_inputs(inputs) for inputs in X])
    
    def _predict_inputs(self, inputs):
        node = self.root
        while node.left:
            if inputs[node.feature_idx] <= node.threshold:
                node = node.left
            else:
                node = node.right
        return node.predicted_class

# ----------------------
# 5. Random Forest from scratch
# ----------------------
class RandomForest:
    def __init__(self, n_trees=5, max_depth=5, min_samples_split=2, max_features=None):
        self.n_trees = n_trees
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.trees = []
        self.max_features = max_features  # number of features to sample per tree
    
    def fit(self, X, y):
        self.trees = []
        for _ in range(self.n_trees):
            # Bootstrap sample
            indices = np.random.choice(len(X), len(X), replace=True)
            X_sample = X[indices]
            y_sample = y[indices]
            # Feature subsample
            if self.max_features is None:
                max_feat = X.shape[1]
            else:
                max_feat = self.max_features
            feat_indices = np.random.choice(X.shape[1], max_feat, replace=False)
            tree = DecisionTree(max_depth=self.max_depth, min_samples_split=self.min_samples_split)
            tree.fit(X_sample[:, feat_indices], y_sample)
            tree.feat_indices = feat_indices  # store feature indices used
            self.trees.append(tree)
    
    def predict(self, X):
        # Majority vote
        tree_preds = np.array([tree.predict(X[:, tree.feat_indices]) for tree in self.trees])
        return np.array([Counter(tree_preds[:,i]).most_common(1)[0][0] for i in range(X.shape[0])])

# ----------------------
# 6. Metrics (from scratch)
# ----------------------
def confusion_matrix_custom(y_true, y_pred):
    TP = np.sum((y_true==1) & (y_pred==1))
    TN = np.sum((y_true==0) & (y_pred==0))
    FP = np.sum((y_true==0) & (y_pred==1))
    FN = np.sum((y_true==1) & (y_pred==0))
    return np.array([[TP, FP],[FN, TN]])

def precision_score_custom(y_true, y_pred):
    TP = np.sum((y_true==1) & (y_pred==1))
    FP = np.sum((y_true==0) & (y_pred==1))
    return TP / (TP + FP + 1e-8)

def recall_score_custom(y_true, y_pred):
    TP = np.sum((y_true==1) & (y_pred==1))
    FN = np.sum((y_true==1) & (y_pred==0))
    return TP / (TP + FN + 1e-8)

def f1_score_custom(y_true, y_pred):
    p = precision_score_custom(y_true, y_pred)
    r = recall_score_custom(y_true, y_pred)
    return 2*p*r / (p+r+1e-8)

# Optional: simple AUC using ranks
def auc_score_custom(y_true, y_score):
    pos = y_score[y_true==1]
    neg = y_score[y_true==0]
    n_pos = len(pos)
    n_neg = len(neg)
    count = 0
    for p in pos:
        count += np.sum(p > neg)
        count += 0.5*np.sum(p == neg)
    return count / (n_pos * n_neg + 1e-8)

# ----------------------
# 7. Train Random Forest
# ----------------------
rf = RandomForest(n_trees=5, max_depth=5, min_samples_split=5, max_features=int(np.sqrt(X_train.shape[1])))
rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)

cm = confusion_matrix_custom(y_test, y_pred)
precision = precision_score_custom(y_test, y_pred)
recall = recall_score_custom(y_test, y_pred)
f1 = f1_score_custom(y_test, y_pred)

print("Confusion Matrix:\n", cm)
print(f"Precision: {precision:.2f}, Recall: {recall:.2f}, F1-score: {f1:.2f}")

# ----------------------
# 8. Feature importance (basic)
# Count how many times each feature is used across all trees
# ----------------------
feature_count = np.zeros(X_train.shape[1])
for tree in rf.trees:
    def traverse(node):
        if node.left:
            feature_count[node.feature_idx] += 1
            traverse(node.left)
            traverse(node.right)
    traverse(tree.root)

for i, col in enumerate(df.drop(columns=[target]).columns):
    print(f"{col}: {feature_count[i]}")



Dataset shape: (6362620, 11)
   step      type    amount     nameOrig  oldbalanceOrg  newbalanceOrig  \
0     1   PAYMENT   9839.64  C1231006815       170136.0       160296.36   
1     1   PAYMENT   1864.28  C1666544295        21249.0        19384.72   
2     1  TRANSFER    181.00  C1305486145          181.0            0.00   
3     1  CASH_OUT    181.00   C840083671          181.0            0.00   
4     1   PAYMENT  11668.14  C2048537720        41554.0        29885.86   

      nameDest  oldbalanceDest  newbalanceDest  isFraud  isFlaggedFraud  
0  M1979787155             0.0             0.0        0               0  
1  M2044282225             0.0             0.0        0               0  
2   C553264065             0.0             0.0        1               0  
3    C38997010         21182.0             0.0        1               0  
4  M1230701703             0.0             0.0        0               0  


KeyError: "['fraud_reported'] not found in axis"